In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import network, compactfilter

### Exercise 1
Verify that the block which had your previous transaction matches the filter for your address.


In [ ]:
# Exercise 1

from block import Block
from compactfilter import GetCFiltersMessage, CFilterMessage
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from network import SimpleNode, GetDataMessage, BLOCK_DATA_TYPE
from script import p2pkh_script
from tx import Tx
block_hash = bytes.fromhex('<block hash from class>')  # CHANGE
block_height = -1  # CHANGE
passphrase = b'<your passphrase here>'  # CHANGE
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(network="signet")
print(addr)
# convert the address to a ScriptPubKey using decode_base58 and p2pkh_script

# connect to signet.programmingbitcoin.com

# complete the handshake

# create a GetCFiltersMessage(start_height, stop_hash) using the block height and block hash

# send the getcfilters message

# wait for the CFilterMessage command

# check that the compact filter's block hash is the same as the block hash


# check if your ScriptPubKey is in the filter


# create a GetDataMessage

# add the BLOCK_DATA_TYPE with the block hash

# send the GetDataMessage

# wait for the Block

# use the get_transactions(script_pubkey) method of Block to get transactions

# print the first one serialized and hexadecimal


### Exercise 2




#### Make [this test](/edit/session8/network.py) pass: `network.py:SimpleNodeTest:test_get_block`

In [ ]:
# Exercise 2

reload(network)
run(network.SimpleNodeTest('test_get_block'))

In [ ]:
from block import Block
from compactfilter import GetCFCheckPointMessage, CFCheckPointMessage, GetCFHeadersMessage, CFHeadersMessage, GetCFiltersMessage, CFilterMessage
from helper import hash256
from network import SimpleNode
num_checkpoints = 20
with open('block_headers.testnet', 'rb') as f:
    headers = [Block.parse_header(f) for _ in range(num_checkpoints * 1000)]
block_hashes = [b.hash() for b in headers]
node = SimpleNode('testnet.programmingbitcoin.com', network="testnet")
node.handshake()
get_cfcheckpoint = GetCFCheckPointMessage(stop_hash=block_hashes[-1])
node.send(get_cfcheckpoint)
cfcheckpoint = node.wait_for(CFCheckPointMessage)
height = 0
for checkpoint in cfcheckpoint.filter_headers:
    get_cfheaders = GetCFHeadersMessage(start_height=height, stop_hash=block_hashes[height+1000])
    node.send(get_cfheaders)
    cfheaders = node.wait_for(CFHeadersMessage)
    if cfheaders.last_header != checkpoint:
        raise RuntimeError(f'checkpoint mismatch {cfheaders.last_header.hex()} vs {checkpoint.hex()}')
    node.send(GetCFiltersMessage(start_height=height, stop_hash=block_hashes[height+999]))
    for i in range(1000):
        fb = node.wait_for(CFilterMessage).filter_bytes
        if hash256(fb) != cfheaders.filter_hashes[i]:
            raise RuntimeError(f'{i}: filter does not match hash {hash256(fb).hex()} vs {cfheaders.filter_hashes[i].hex()}')
    height += 1000
print(cfheaders.last_header.hex())

### Exercise 3
You have been sent some unknown number of sats to your address on signet.

Send all of it back (minus fees) to `mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv` using only the networking protocol.

This should be a many input, 1 output transaction.

Turn on logging in `SimpleNode` if you need to debug


In [ ]:
# Exercise 3

from block import Block
from compactfilter import GetCFiltersMessage, CFilterMessage
from ecc import PrivateKey
from helper import decode_base58, hash160, hash256, little_endian_to_int
from network import GetHeadersMessage, HeadersMessage, SimpleNode, BLOCK_DATA_TYPE
from script import p2pkh_script
from tx import Tx, TxIn, TxOut
start_block_hex = '<insert from class>'  # CHANGE
start_block = bytes.fromhex(start_block_hex)
start_height = -1  # CHANGE
end_block = b'\x00' * 32
passphrase = b'<get from session 2>'  # CHANGE
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(network="signet")
print(addr)
h160 = decode_base58(addr)
my_script_pubkey = p2pkh_script(h160)
target_address = 'mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv'
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 200  # fee in satoshis
# connect to signet.programmingbitcoin.com in signet mode

# complete the handshake

# create GetHeadersMessage with the start_block as the start_block and end_block as the end block

# send the GetHeadersMessage

# wait for the headers message

# check that the headers are valid


# get the 20th hash (index 19) from the header.headers array

# create a GetCFiltersMessage

# send the GetCFiltersMessage

# loop 100 times

    # wait for the CFilterMessage

    # check to see if your ScriptPubKey is in the filter

        # set block_hash to cfilter's block hash and break


        break
# get the block object using the get_block method of node

# initialize the utxos array

# grab the txs from the block using get_transactions(my_script_pubkey) method

# there should be one transaction
if len(txs) != 1:
    raise RuntimeError("incorrect number of transactions")
# set utxos to the tx's utxos for our address using find_utxos(addr) method of the first tx

# there should be one utxo
if len(utxos) != 1:
    raise RuntimeError("incorrect number of utxos")
# initialize the tx_ins array

# prev_tx, prev_index, prev_amount are what we get in the first utxo

# create TxIn and add to array

# calculate the output amount (prev_amount - fee)

# create TxOut

# create transaction on signet

# sign the only input in the tx

# print the tx's id

# send this signed transaction on the network
